In [28]:
import sqlite3
import pandas as pd

In [29]:
import re

In [30]:
pd.set_option("display.max_colwidth",500)
pd.set_option("display.max_rows",500)
features_to_show = ["crawl_id", "func_name", "top_level_url", "symbol", "operation", "value", "arguments"]

In [31]:
DB = 'sample_2018-06_1m_stateless_census_crawl.sqlite'

In [32]:
con = sqlite3.connect(DB)

In [33]:
con.row_factory = sqlite3.Row
cur = con.cursor()
js = pd.read_sql_query("SELECT * FROM javascript", con)

In [34]:
# js

In [35]:
js[js.operation == "call"].symbol.value_counts().head(15)

window.Storage.getItem                  46851
window.Storage.setItem                  18104
window.Storage.removeItem               13812
CanvasRenderingContext2D.fill            7258
CanvasRenderingContext2D.save            7074
CanvasRenderingContext2D.restore         7070
HTMLCanvasElement.getContext             4208
window.Storage.key                       3689
CanvasRenderingContext2D.measureText     3103
CanvasRenderingContext2D.stroke          2393
CanvasRenderingContext2D.arc             1558
CanvasRenderingContext2D.fillRect        1371
CanvasRenderingContext2D.clip            1327
CanvasRenderingContext2D.fillText         858
CanvasRenderingContext2D.rotate           537
Name: symbol, dtype: int64

In [36]:
js[(js.operation == "call") &
   (js.symbol == "CanvasRenderingContext2D.fillText")
  ].arguments.value_counts().head(10)

{"0":"Cwm fjordbank glyphs vext quiz, 😃","1":4,"2":45}    74
{"0":"Cwm fjordbank glyphs vext quiz, 😃","1":2,"2":15}    74
{"0":"!image!","1":4,"2":17}                              39
{"0":"!image!","1":2,"2":15}                              39
{"0":"!H71JCaj)]# 1@#","1":4,"2":8}                       19
{"0":"Soft Ruddy Foothold 2","1":2,"2":2}                 19
{"0":"🇺🇳","1":0,"2":0}                                    18
{"0":"🇺​🇳","1":0,"2":0}                                   18
{"0":"09:30","1":5,"2":130}                               14
{"0":"🕴​♀️","1":0,"2":0}                                  14
Name: arguments, dtype: int64

<b> Criteria 1 from Englehardt & Narayanan, 2016: </b>

To be a fingerprinting candidate: "1.The canvas element’s height and width properties must
not be set below 16 px."

In [37]:
def width_height_finder(data):
    w = re.compile('[^(line)]width', re.IGNORECASE)
    h = re.compile('[^(line)]height', re.IGNORECASE)
    ww = w.search(data)
    hh = h.search(data)
    if ww!= None:
        return data
    elif hh!=None:
        return data

In [38]:
l = list(map(width_height_finder, list(js[js.operation == "set"].symbol)))
l = [x for x in l if x is not None]

In [39]:
set(l)

{'HTMLCanvasElement.height', 'HTMLCanvasElement.width'}

In [40]:
MIN_CANVAS_IMAGE_WIDTH = 16
MIN_CANVAS_IMAGE_HEIGHT = 16


canvas_size = js[js.symbol.isin(["HTMLCanvasElement.width", "HTMLCanvasElement.height"])].value.astype(float)
index_size_filter = canvas_size.where(canvas_size > MIN_CANVAS_IMAGE_HEIGHT).dropna().index

In [41]:
index_size_filter

Int64Index([   638,    639,   1138,   1139,   1142,   1143,   1192,   1193,
              1241,   1242,
            ...
            501091, 501092, 501186, 501187, 501190, 501191, 501196, 501197,
            501200, 501201],
           dtype='int64', length=7559)

In [42]:
# js.iloc[index_size_filter][features_to_show]

<b> Criteria 2 from Englehardt & Narayanan, 2016: </b> 

To be a fingerprinting candidate: "2.Text must be written to canvas with least two colors or
at least 10 distinct characters."

In [43]:
# js.symbol.value_counts()

In [44]:
# js[js.symbol == "CanvasRenderingContext2D.strokeStyle"]

In [45]:
import json 

def count_distinct_letters(text):
    if text!=None:
        t = json.loads(text, encoding='utf-8')['0']
        if not isinstance(t, int):
            return len(set(t))

In [46]:
CANVAS_WRITE_FUNCS = [
    "CanvasRenderingContext2D.fillText",
    "CanvasRenderingContext2D.strokeText",
    "CanvasRenderingContext2D.fill"
    ]

In [47]:
text = js[js.symbol.isin(CANVAS_WRITE_FUNCS)].arguments.dropna()
indexx = text.index
text = map(count_distinct_letters, text) 
res = pd.Series(text, index= indexx)
index_count_filter = res.where(res>=10).dropna().index

In [48]:
# js.iloc[index_count_filter][features_to_show]

<b> Criteria 3 from Englehardt & Narayanan, 2016 </b>

To be a fingerprinting candidate: "3. The script should not call the save, restore, or addEventListener
methods of the rendering context."

In [49]:
CANVAS_FP_DO_NOT_CALL_LIST = ["CanvasRenderingContext2D.save",
                              "CanvasRenderingContext2D.restore",
                              "HTMLCanvasElement.addEventListener"]

In [50]:
# js[js.operation == "call"][js.arguments!=None].arguments.value_counts().head(15)
index_call_filter = js[~js.symbol.isin(CANVAS_FP_DO_NOT_CALL_LIST)].index
index_call_filter

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            501197, 501198, 501199, 501200, 501201, 501202, 501203, 501204,
            501205, 501206],
           dtype='int64', length=486982)

In [51]:
# js.iloc[index_call_filter][features_to_show]

In [52]:
# js[js.crawl_id==14].symbol.isin(CANVAS_FP_DO_NOT_CALL_LIST)

<b>Criteria 4 from Englehardt & Narayanan, 2016: </b>

To be a fingerprinting candidate: "4. The script extracts an image with toDataURL or with a
single call to getImageData that specifies an area with a
minimum size of 16px × 16px"

In [53]:
# js.symbol.value_counts()

In [54]:
CANVAS_READ_FUNCS = [
    "HTMLCanvasElement.toDataURL",
    "CanvasRenderingContext2D.getImageData"
    ]

In [55]:
# js[js.symbol.isin(CANVAS_READ_FUNCS)].arguments.dropna()

In [56]:
def check_dimensions(image):
    if image!=None:
        img = json.loads(image, encoding='utf-8')
        if len(img)>3 and img['2']>=16 and img['3']>=16:
            return image

In [57]:
image = js[js.symbol.isin(CANVAS_READ_FUNCS)].arguments
indexx = image.index
image = map(check_dimensions, image)
res = pd.Series(image, index = indexx).dropna()
index_image_filter = res.index
index_image_filter

Int64Index([ 55807,  65731,  94199,  94241, 107361, 107375, 107389, 107403,
            107417, 107433,
            ...
            339166, 339172, 339177, 339182, 339187, 339192, 339197, 339202,
            339207, 443435],
           dtype='int64', length=148)

In [58]:
js.iloc[index_image_filter][features_to_show]

,crawl_id,func_name,top_level_url,symbol,operation,value,arguments
55807,13,i._update,https://www.vice.com/en_us,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":32,""3"":32}"
65731,9,[158]</n.getImage/</r.getData,https://www.washingtonpost.com/?noredirect=on,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":76,""3"":76}"
94199,8,a.exports/p<,http://www.businessinsider.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":200,""3"":200}"
94241,8,P.prototype.C/</h<,http://www.businessinsider.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":200,""3"":200}"
107361,9,b.applyFilter,https://www.webtretho.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":302,""3"":302}"
107375,9,b.applyFilter,https://www.webtretho.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":151,""3"":151}"
107389,9,b.applyFilter,https://www.webtretho.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":97,""3"":97}"
107403,9,b.applyFilter,https://www.webtretho.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":178,""3"":178}"
107417,9,b.applyFilter,https://www.webtretho.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":140,""3"":140}"
107433,9,b.applyFilter,https://www.webtretho.com/,CanvasRenderingContext2D.getImageData,call,,"{""0"":0,""1"":0,""2"":140,""3"":140}"


At least 2 criteria

In [59]:
lst = []

In [60]:
for i in range(len(js)):
    if i in index_call_filter and i in index_count_filter:
        lst.append(js.iloc[i])
    elif i in index_call_filter and i in index_size_filter:
        lst.append(js.iloc[i])
    elif i in index_call_filter and i in index_image_filter:
        lst.append(js.iloc[i])
    elif i in index_count_filter and i in index_size_filter:
        lst.append(js.iloc[i])
    elif i in index_count_filter and i in index_image_filter:
        lst.append(js.iloc[i])
    elif i in index_image_filter and i in index_size_filter:
        lst.append(js.iloc[i])

In [61]:
df = pd.DataFrame(lst)
len(df)

8122

In [68]:
fp_crawl_id = []
for i in set(js.crawl_id):
    cond1 = cond2 = cond3 = cond4 = False
    for j in js[js.crawl_id==i].index:
        if j in index_size_filter:
            cond1 = True
        if j in index_count_filter:
            cond2 = True
        if j in index_call_filter:
            cond3 = True
        if j in index_image_filter:
            cond4 = True
    if cond1 and cond2 and cond3 and cond4:
        fp_crawl_id.append(i)

In [69]:
len(fp_crawl_id)

8

Each criterion separately

In [34]:
# lst1 = []
# lst2 = []
# lst3 = []
# lst4 = []

In [35]:
# for i in range(len(js)):
#     if i in index_size_filter:
#         lst1.append(js.iloc[i])
#     elif i in index_count_filter:
#         lst2.append(js.iloc[i])
#     elif i in index_call_filter:
#         lst3.append(js.iloc[i])
#     elif i in index_image_filter:
#         lst4.append(js.iloc[i])

In [36]:
# crt1 = pd.DataFrame(lst1)
# crt2 = pd.DataFrame(lst2)
# crt3 = pd.DataFrame(lst3)
# crt4 = pd.DataFrame(lst4)

In [37]:
# crt1[features_to_show]

All four criteria

In [38]:
# lst = []

In [39]:
# for i in range(len(js)):
#     if i in index_call_filter and i in index_count_filter and i in index_size_filter and i in index_image_filter:
#         lst.append(js.iloc[i])

In [40]:
# df = pd.DataFrame(lst)
# len(df)